# CNN

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Processamento

In [126]:
df = pd.read_csv(r"C:\Users\annab\OneDrive\Documentos\GitHub\Colabs\bases\data_prema.csv")

In [127]:
df = df [(df['QTDFILVIVO'] < 12)]
df = df [(df['QTDFILMORT'] < 8)]

In [128]:
df = df[['ESTCIVMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GESTACAO', 'GRAVIDEZ', 'SEXO', 'RACACORMAE', 
         'QTDGESTANT', 'QTDPARTNOR', 'QTDPARTCES', 'ESCMAEAGR1', 'MESPRENAT', 'IDADEMAE']]

In [129]:
faixas_etarias = [0, 19, 24, 29, 34, 99]
rotulos_etarias = ['0-19', '20-24', '25-29', '30-34', '35-99']

df['FAIXA_ETARIA'] = pd.cut(df['IDADEMAE'], bins=faixas_etarias, labels=rotulos_etarias, right=False)
mapeamento_faixa_etaria = {'0-19': 0, '20-24': 1, '25-29': 2, '30-34': 3, '35-99': 4}
df['FAIXA_ETARIA_INT'] = df['FAIXA_ETARIA'].map(mapeamento_faixa_etaria)
print(df[['IDADEMAE', 'FAIXA_ETARIA', 'FAIXA_ETARIA_INT']])
df = df.drop(['FAIXA_ETARIA', 'IDADEMAE'], axis= 1)

        IDADEMAE FAIXA_ETARIA FAIXA_ETARIA_INT
0             29        30-34                3
1             37        35-99                4
2             16         0-19                0
3             22        20-24                1
4             33        30-34                3
...          ...          ...              ...
526363        23        20-24                1
526364        24        25-29                2
526365        30        30-34                3
526366        24        25-29                2
526367        23        20-24                1

[522055 rows x 3 columns]


In [130]:
df = df.dropna()

In [131]:
df['CLASSE'] = df['GESTACAO'].apply(lambda x: 1 if x in [1, 2, 3, 4] else 0)
print(df['CLASSE'].value_counts())

CLASSE
0    440429
1     54707
Name: count, dtype: int64


In [132]:
df = df.drop(columns = ['GESTACAO'])

In [133]:
df_balanceado_1 = df[df['CLASSE'] == 1].sample(n=16690, random_state=42)
df_balanceado_0 = df[df['CLASSE'] == 0].sample(n=16690, random_state=42)

df_teste = pd.concat([df_balanceado_1, df_balanceado_0])
df = df.drop(df_teste.index, errors='ignore') 

df_teste.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

print("DataFrame de Teste:")
print(df_teste)


DataFrame de Teste:
       ESTCIVMAE  QTDFILVIVO  QTDFILMORT  GRAVIDEZ  SEXO  RACACORMAE  \
0            2.0         1.0         1.0       1.0     1         4.0   
1            1.0         1.0         1.0       2.0     2         1.0   
2            1.0         1.0         0.0       1.0     2         4.0   
3            1.0         1.0         0.0       1.0     2         1.0   
4            2.0         1.0         1.0       1.0     2         2.0   
...          ...         ...         ...       ...   ...         ...   
33375        2.0         1.0         1.0       1.0     1         4.0   
33376        1.0         0.0         0.0       1.0     2         4.0   
33377        1.0         0.0         0.0       1.0     2         4.0   
33378        2.0         0.0         1.0       1.0     1         4.0   
33379        5.0         1.0         0.0       1.0     1         4.0   

       QTDGESTANT  QTDPARTNOR  QTDPARTCES  ESCMAEAGR1  MESPRENAT  \
0             2.0         1.0         0.0      

# Modelo desbalanceado

In [40]:
df.columns

Index(['IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'QTDFILMORT',
       'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'SEXO', 'RACACOR', 'ESCMAE2010',
       'SERIESCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR', 'QTDPARTCES',
       'SEMAGESTAC', 'MESPRENAT', 'ESCMAEAGR1', 'PARIDADE', 'CLASSE'],
      dtype='object')

In [41]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

x = df.drop('CLASSE', axis=1).values  
y = df['CLASSE'].values               

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


Epoch 1/20


c:\Users\annab\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11515/11515 ━━━━━━━━━━━━━━━━━━━━ 47s 4ms/step - accuracy: 0.8906 - loss: 0.4241 - val_accuracy: 0.8921 - val_loss: 0.3421
Epoch 2/20
11515/11515 ━━━━━━━━━━━━━━━━━━━━ 47s 4ms/step - accuracy: 0.8918 - loss: 0.3428 - val_accuracy: 0.8921 - val_loss: 0.3421
Epoch 3/20
11515/11515 ━━━━━━━━━━━━━━━━━━━━ 51s 4ms/step - accuracy: 0.8927 - loss: 0.3407 - val_accuracy: 0.8921 - val_loss: 0.3421
Epoch 4/20
11515/11515 ━━━━━━━━━━━━━━━━━━━━ 62s 5ms/step - accuracy: 0.8911 - loss: 0.3443 - val_accuracy: 0.8921 - val_loss: 0.3421
Epoch 5/20
11515/11515 ━━━━━━━━━━━━━━━━━━━━ 53s 5ms/step - accuracy: 0.8917 - loss: 0.3429 - val_accuracy: 0.8921 - val_loss: 0.3421
Epoch 6/20
11515/11515 ━━━━━━━━━━━━━━━━━━━━ 47s 4ms/step - accuracy: 0.8905 - loss: 0.3454 - val_accuracy: 0.8921 - val_loss: 0.3421
Epoch 7/20
11515/11515 ━━━━━━━━━━━━━━━━━━━━ 51s 4ms/step - accuracy: 0.8922 - loss: 0.3420 - val_accuracy: 0.8921 - val_loss: 0.3421
Epoch 8/20
11515/11515 ━━━━━━━━━━━━━━━━━━━━ 50s 4ms/step - accuracy: 0.8910 - lo

c:\Users\annab\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\annab\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\annab\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.89      1.00      0.94    140872
           1       0.00      0.00      0.00     17039

    accuracy                           0.89    157911
   macro avg       0.45      0.50      0.47    157911
weighted avg       0.80      0.89      0.84    157911



# Undersampling

In [56]:
from sklearn.utils import resample

classe_0 = df[df['CLASSE'] == 0]
classe_1 = df[df['CLASSE'] == 1]

min_size = min(len(classe_0), len(classe_1))
classe_0_downsampled = resample(classe_0, replace=False, n_samples=min_size, random_state=42)
classe_1_downsampled = resample(classe_1, replace=False, n_samples=min_size, random_state=42)

df_balanced = pd.concat([classe_0_downsampled, classe_1_downsampled])

print(df_balanced['CLASSE'].value_counts())


CLASSE
0    56987
1    56987
Name: count, dtype: int64


In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

x = df_balanced.drop('CLASSE', axis=1).values  
y = df_balanced['CLASSE'].values               

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


Epoch 1/20


c:\Users\annab\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2494/2494 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.5003 - loss: 0.6932 - val_accuracy: 0.5030 - val_loss: 0.6931
Epoch 2/20
2494/2494 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.5010 - loss: 0.6932 - val_accuracy: 0.4970 - val_loss: 0.6932
Epoch 3/20
2494/2494 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.5006 - loss: 0.6932 - val_accuracy: 0.4970 - val_loss: 0.6932
Epoch 4/20
2494/2494 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.4959 - loss: 0.6932 - val_accuracy: 0.5030 - val_loss: 0.6931
Epoch 5/20
2494/2494 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.4958 - loss: 0.6932 - val_accuracy: 0.4970 - val_loss: 0.6932
Epoch 6/20
2494/2494 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.5040 - loss: 0.6931 - val_accuracy: 0.5030 - val_loss: 0.6931
Epoch 7/20
2494/2494 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.4970 - loss: 0.6932 - val_accuracy: 0.4970 - val_loss: 0.6932
Epoch 8/20
2494/2494 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.4996 - loss: 0.6932 - val_a

c:\Users\annab\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\annab\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\annab\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

# Hybridsampling 2x

In [134]:
df_normal = df[df['CLASSE'] == 0]
df_prema = df[df['CLASSE'] == 1]

In [135]:
undersampled_df = df_normal.sample(n=77228, random_state=42)
print("Tamanho do DataFrame undersampled:", undersampled_df.shape)

Tamanho do DataFrame undersampled: (77228, 13)


In [136]:
df_unificado = pd.concat([df_prema,undersampled_df])
df_unificado

,ESTCIVMAE,QTDFILVIVO,QTDFILMORT,GRAVIDEZ,SEXO,RACACORMAE,QTDGESTANT,QTDPARTNOR,QTDPARTCES,ESCMAEAGR1,MESPRENAT,FAIXA_ETARIA_INT,CLASSE
0,2.0,0.0,0.0,1.0,2,4.0,0.0,0.0,0.0,6.0,1.0,4,1
60,1.0,0.0,0.0,1.0,2,4.0,0.0,0.0,0.0,7.0,2.0,3,1
63,2.0,2.0,0.0,1.0,1,4.0,2.0,0.0,2.0,8.0,1.0,4,1
107,2.0,2.0,0.0,1.0,2,4.0,2.0,1.0,1.0,7.0,2.0,2,1
156,1.0,0.0,0.0,1.0,2,4.0,0.0,0.0,0.0,5.0,2.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89735,2.0,0.0,0.0,1.0,2,1.0,0.0,0.0,0.0,8.0,2.0,4,0
96312,2.0,5.0,1.0,1.0,2,2.0,6.0,5.0,0.0,2.0,3.0,3,0
190145,1.0,0.0,0.0,1.0,2,4.0,0.0,0.0,0.0,5.0,3.0,0,0
397880,1.0,2.0,2.0,1.0,1,4.0,4.0,0.0,2.0,5.0,2.0,2,0


In [137]:
import pandas as pd
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=5, n_jobs=None)

x = df_unificado.drop('CLASSE', axis=1)
y = df_unificado['CLASSE']

X_resampled, y_resampled = smote.fit_resample(x, y)
df_balanceado = pd.DataFrame(X_resampled, columns=x.columns)
df_balanceado['CLASSE'] = y_resampled

print(df_balanceado['CLASSE'].value_counts())

CLASSE
1    77228
0    77228
Name: count, dtype: int64


In [138]:
df_unificado.columns

Index(['ESTCIVMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'SEXO',
       'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR', 'QTDPARTCES', 'ESCMAEAGR1',
       'MESPRENAT', 'FAIXA_ETARIA_INT', 'CLASSE'],
      dtype='object')

In [139]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

X_train = df_unificado.drop('CLASSE', axis=1).values  
y_train = df_unificado['CLASSE'].values               

X_test = df_teste.drop('CLASSE', axis=1).values  
y_test = df_teste['CLASSE'].values    

model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


Epoch 1/20


c:\Users\annab\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3602/3602 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.6732 - loss: 0.6329 - val_accuracy: 0.5481 - val_loss: 0.7048
Epoch 2/20
3602/3602 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.6919 - loss: 0.6158 - val_accuracy: 0.5473 - val_loss: 0.7301
Epoch 3/20
3602/3602 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.6935 - loss: 0.6132 - val_accuracy: 0.5482 - val_loss: 0.7209
Epoch 4/20
3602/3602 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.6926 - loss: 0.6137 - val_accuracy: 0.5477 - val_loss: 0.7189
Epoch 5/20
3602/3602 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.6950 - loss: 0.6116 - val_accuracy: 0.5489 - val_loss: 0.7132
Epoch 6/20
3602/3602 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.6949 - loss: 0.6119 - val_accuracy: 0.5493 - val_loss: 0.7168
Epoch 7/20
3602/3602 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.6931 - loss: 0.6129 - val_accuracy: 0.5488 - val_loss: 0.7213
Epoch 8/20
3602/3602 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.6924 - loss: 0.6129 - val

In [149]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

conf_matrix = confusion_matrix(y_test, y_pred)
sensitivity = recall

print("Metrics Report:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Metrics Report:
Accuracy: 0.5478
Precision: 0.9144
Recall (Sensitivity): 0.1056
F1-Score: 0.1893


# Hybridsampling 3x

In [142]:
df_normal = df[df['CLASSE'] == 0]
df_prema = df[df['CLASSE'] == 1]

In [143]:
undersampled_df = df_normal.sample(n=115842, random_state=42)
print("Tamanho do DataFrame undersampled:", undersampled_df.shape)

Tamanho do DataFrame undersampled: (115842, 13)


In [144]:
df_unificado = pd.concat([df_prema,undersampled_df])
df_unificado

,ESTCIVMAE,QTDFILVIVO,QTDFILMORT,GRAVIDEZ,SEXO,RACACORMAE,QTDGESTANT,QTDPARTNOR,QTDPARTCES,ESCMAEAGR1,MESPRENAT,FAIXA_ETARIA_INT,CLASSE
0,2.0,0.0,0.0,1.0,2,4.0,0.0,0.0,0.0,6.0,1.0,4,1
60,1.0,0.0,0.0,1.0,2,4.0,0.0,0.0,0.0,7.0,2.0,3,1
63,2.0,2.0,0.0,1.0,1,4.0,2.0,0.0,2.0,8.0,1.0,4,1
107,2.0,2.0,0.0,1.0,2,4.0,2.0,1.0,1.0,7.0,2.0,2,1
156,1.0,0.0,0.0,1.0,2,4.0,0.0,0.0,0.0,5.0,2.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19032,1.0,1.0,1.0,1.0,2,4.0,2.0,0.0,1.0,6.0,99.0,4,0
90713,5.0,1.0,0.0,1.0,1,4.0,1.0,1.0,0.0,3.0,1.0,1,0
423979,5.0,1.0,0.0,1.0,1,2.0,1.0,0.0,1.0,12.0,2.0,3,0
165816,5.0,0.0,0.0,1.0,2,4.0,0.0,0.0,0.0,7.0,2.0,2,0


In [145]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

X_train = df_unificado.drop('CLASSE', axis=1).values  
y_train = df_unificado['CLASSE'].values               

X_test = df_teste.drop('CLASSE', axis=1).values  
y_test = df_teste['CLASSE'].values    

model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


Epoch 1/20


c:\Users\annab\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4809/4809 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.7541 - loss: 0.5608 - val_accuracy: 0.5443 - val_loss: 0.8108
Epoch 2/20
4809/4809 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.7645 - loss: 0.5442 - val_accuracy: 0.5421 - val_loss: 0.7894
Epoch 3/20
4809/4809 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.7691 - loss: 0.5387 - val_accuracy: 0.5494 - val_loss: 0.7931
Epoch 4/20
4809/4809 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.7686 - loss: 0.5393 - val_accuracy: 0.5472 - val_loss: 0.7763
Epoch 5/20
4809/4809 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.7677 - loss: 0.5402 - val_accuracy: 0.5490 - val_loss: 0.7799
Epoch 6/20
4809/4809 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.7666 - loss: 0.5411 - val_accuracy: 0.5476 - val_loss: 0.7967
Epoch 7/20
4809/4809 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.7680 - loss: 0.5400 - val_accuracy: 0.5475 - val_loss: 0.7840
Epoch 8/20
4809/4809 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.7696 - loss: 0.5380 - val

In [148]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

conf_matrix = confusion_matrix(y_test, y_pred)

sensitivity = recall

print("Metrics Report:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Metrics Report:
Accuracy: 0.5478
Precision: 0.9144
Recall (Sensitivity): 0.1056
F1-Score: 0.1893


# Hybridsampling 4x

In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

def create_model(optimizer='adam', dropout_rate=0.3, neurons=128):
    model = Sequential([
        Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(dropout_rate),
        Dense(neurons, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')  
    ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {
    'batch_size': [16, 32],
    'epochs': [10, 20, 50, 100],
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.2, 0.3, 0.4],
    'neurons': [32, 64, 128, 256, 512],
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5)

grid_result = grid.fit(X_train, y_train)

print(f"Best parameters found: {grid_result.best_params_}")
print(f"Best score: {grid_result.best_score_:.4f}")

best_model = grid_result.best_estimator_
loss, accuracy = best_model.model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

y_pred = (best_model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))
